In [40]:
import json
import pandas as pd
import numpy as np
import os
from PIL import Image

import warnings

# SettingWithCopyWarning 경고를 아예 무시(ignore)하도록 설정
warnings.simplefilter(action='ignore', category=Warning)
# 1. 파일 경로 설정
json_path = '/home/bongo/porter_notebook/research/qwen3/3B_exo_selection_all.json'

In [3]:

with open(json_path, 'r',encoding='utf-16') as f:
    data = json.load(f)
    
# 3. 데이터 평탄화 (Flattening)
# 목표 구조: [{'pair_key': 'hold$axe', 'file_name': 'abc.jpg', 'beat': -3.5, ...}, ...]
rows = []

for pair_key, files_dict in data.items():
    for file_name, scores_dict in files_dict.items():
        # 기본 메타데이터 생성
        row_data = {
            'pair_key': pair_key,   # 예: 'hold$axe'
            'file_name': file_name  # 예: 'hold_axe_000384.jpg'
        }
        # 액션 점수 딕셔너리를 합침 (Action Score 컬럼들 추가)
        row_data.update(scores_dict)
        
        rows.append(row_data)

# 4. DataFrame 생성
df = pd.DataFrame(rows)
df[['action', 'object_name']] = df['pair_key'].str.split('$', expand=True)
AGD20K_PATH = "/home/DATA/AGD20K"
df['exo_path'] = df.apply(lambda x: f"{AGD20K_PATH}/Seen/trainset/exocentric/{x['action']}/{x['object_name']}/{x['file_name']}",axis=1)
df

,pair_key,file_name,beat,boxing,brush_with,carry,catch,cut,cut_with,drag,...,take_photo,talk_on,text_on,throw,type_on,wash,write,action,object_name,exo_path
0,beat$drum,beat_drum_000099.jpg,23.125000,17.0000,16.1250,9.062500,13.3125,13.5625,13.5625,18.1250,...,11.5000,7.343750,8.12500,19.5000,10.2500,6.906250,8.6250,beat,drum,/home/DATA/AGD20K/Seen/trainset/exocentric/bea...
1,beat$drum,beat_drum_000247.jpg,21.375000,14.8750,14.4375,10.187500,10.4375,10.3125,10.3125,16.2500,...,12.5625,9.312500,7.25000,16.2500,10.5625,7.687500,10.5000,beat,drum,/home/DATA/AGD20K/Seen/trainset/exocentric/bea...
2,beat$drum,beat_drum_000277.jpg,18.625000,16.2500,14.4375,9.812500,11.5625,12.2500,12.2500,15.9375,...,8.6250,6.000000,8.06250,16.2500,9.1250,4.125000,5.9375,beat,drum,/home/DATA/AGD20K/Seen/trainset/exocentric/bea...
3,beat$drum,beat_drum_000389.jpg,20.000000,14.8750,13.6875,8.687500,9.5625,9.1875,9.1875,15.3750,...,8.9375,7.187500,7.15625,15.7500,8.5000,3.656250,4.8125,beat,drum,/home/DATA/AGD20K/Seen/trainset/exocentric/bea...
4,beat$drum,beat_drum_000424.jpg,24.000000,14.9375,15.1875,9.312500,9.8125,9.8750,9.8750,15.7500,...,12.1875,7.062500,6.50000,16.2500,9.8750,6.187500,10.2500,beat,drum,/home/DATA/AGD20K/Seen/trainset/exocentric/bea...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20055,write$pen,write_pen_013769.jpg,-1.039062,11.0625,14.8750,2.156250,9.7500,12.7500,12.7500,13.6875,...,5.8750,0.120605,14.06250,12.0000,11.3750,3.140625,24.1250,write,pen,/home/DATA/AGD20K/Seen/trainset/exocentric/wri...
20056,write$pen,write_pen_013824.jpg,1.132812,9.8750,14.8125,4.468750,10.5625,12.1250,12.1250,12.2500,...,10.2500,2.171875,15.43750,11.4375,13.6875,5.937500,28.7500,write,pen,/home/DATA/AGD20K/Seen/trainset/exocentric/wri...
20057,write$pen,write_pen_013836.jpg,-0.550781,10.1875,15.5625,3.328125,10.8750,12.7500,12.7500,13.5000,...,5.8750,0.433594,14.25000,12.0000,10.4375,3.734375,25.5000,write,pen,/home/DATA/AGD20K/Seen/trainset/exocentric/wri...
20058,write$pen,write_pen_013937.jpg,-0.023315,10.1250,15.3750,2.984375,9.8750,11.8125,11.8125,13.6875,...,5.6250,0.898438,14.25000,11.3750,11.7500,3.328125,24.8750,write,pen,/home/DATA/AGD20K/Seen/trainset/exocentric/wri...


In [56]:
df.pair_key.unique()

array(['beat$drum', 'boxing$punching_bag', 'brush_with$toothbrush',
       'carry$skateboard', 'carry$skis', 'carry$snowboard',
       'carry$surfboard', 'catch$frisbee', 'catch$rugby_ball',
       'catch$soccer_ball', 'cut$apple', 'cut$banana', 'cut$carrot',
       'cut$orange', 'cut_with$knife', 'cut_with$scissors',
       'drag$suitcase', 'drink_with$bottle', 'drink_with$cup',
       'drink_with$wine_glass', 'eat$apple', 'eat$banana', 'eat$broccoli',
       'eat$carrot', 'eat$hot_dog', 'eat$orange', 'hit$axe',
       'hit$baseball_bat', 'hit$hammer', 'hit$tennis_racket', 'hold$axe',
       'hold$badminton_racket', 'hold$baseball_bat', 'hold$book',
       'hold$bottle', 'hold$bowl', 'hold$cup', 'hold$fork',
       'hold$frisbee', 'hold$golf_clubs', 'hold$hammer', 'hold$knife',
       'hold$scissors', 'hold$skateboard', 'hold$skis', 'hold$snowboard',
       'hold$suitcase', 'hold$surfboard', 'hold$tennis_racket',
       'hold$toothbrush', 'hold$wine_glass', 'jump$skateboard',
       '

In [5]:
def get_image_area_from_dir(dir_path):
    """
    폴더 경로를 입력받아 내부의 첫 번째 이미지를 찾고
    (너비, 높이, 넓이)를 반환합니다.
    """
    # 1. 경로 존재 여부 확인
    if not os.path.exists(dir_path):
        return None, None, None
    
    # 2. 이미지 확장자 필터링
    valid_exts = ('.jpg', '.jpeg', '.png', '.bmp', '.PNG', '.JPG')
    

        
    # 3. 첫 번째 이미지 열기 (파일 전체를 로드하지 않고 헤더만 읽어서 빠름)
    first_img_path = os.path.join(dir_path)
    with Image.open(first_img_path) as img:
        w, h = img.size
        area = w * h
        return w, h, area
            
        
df[['exo_w', 'exo_h', 'exo_area']] = df['exo_path'].apply(
    lambda path: pd.Series(get_image_area_from_dir(path))
)
df

,pair_key,file_name,beat,boxing,brush_with,carry,catch,cut,cut_with,drag,...,throw,type_on,wash,write,action,object_name,exo_path,exo_w,exo_h,exo_area
0,beat$drum,beat_drum_000099.jpg,23.125000,17.0000,16.1250,9.062500,13.3125,13.5625,13.5625,18.1250,...,19.5000,10.2500,6.906250,8.6250,beat,drum,/home/DATA/AGD20K/Seen/trainset/exocentric/bea...,320,240,76800
1,beat$drum,beat_drum_000247.jpg,21.375000,14.8750,14.4375,10.187500,10.4375,10.3125,10.3125,16.2500,...,16.2500,10.5625,7.687500,10.5000,beat,drum,/home/DATA/AGD20K/Seen/trainset/exocentric/bea...,640,480,307200
2,beat$drum,beat_drum_000277.jpg,18.625000,16.2500,14.4375,9.812500,11.5625,12.2500,12.2500,15.9375,...,16.2500,9.1250,4.125000,5.9375,beat,drum,/home/DATA/AGD20K/Seen/trainset/exocentric/bea...,640,426,272640
3,beat$drum,beat_drum_000389.jpg,20.000000,14.8750,13.6875,8.687500,9.5625,9.1875,9.1875,15.3750,...,15.7500,8.5000,3.656250,4.8125,beat,drum,/home/DATA/AGD20K/Seen/trainset/exocentric/bea...,430,640,275200
4,beat$drum,beat_drum_000424.jpg,24.000000,14.9375,15.1875,9.312500,9.8125,9.8750,9.8750,15.7500,...,16.2500,9.8750,6.187500,10.2500,beat,drum,/home/DATA/AGD20K/Seen/trainset/exocentric/bea...,640,426,272640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20055,write$pen,write_pen_013769.jpg,-1.039062,11.0625,14.8750,2.156250,9.7500,12.7500,12.7500,13.6875,...,12.0000,11.3750,3.140625,24.1250,write,pen,/home/DATA/AGD20K/Seen/trainset/exocentric/wri...,640,426,272640
20056,write$pen,write_pen_013824.jpg,1.132812,9.8750,14.8125,4.468750,10.5625,12.1250,12.1250,12.2500,...,11.4375,13.6875,5.937500,28.7500,write,pen,/home/DATA/AGD20K/Seen/trainset/exocentric/wri...,640,428,273920
20057,write$pen,write_pen_013836.jpg,-0.550781,10.1875,15.5625,3.328125,10.8750,12.7500,12.7500,13.5000,...,12.0000,10.4375,3.734375,25.5000,write,pen,/home/DATA/AGD20K/Seen/trainset/exocentric/wri...,640,426,272640
20058,write$pen,write_pen_013937.jpg,-0.023315,10.1250,15.3750,2.984375,9.8750,11.8125,11.8125,13.6875,...,11.3750,11.7500,3.328125,24.8750,write,pen,/home/DATA/AGD20K/Seen/trainset/exocentric/wri...,640,426,272640


In [57]:
df.pair_key.unique()

array(['beat$drum', 'boxing$punching_bag', 'brush_with$toothbrush',
       'carry$skateboard', 'carry$skis', 'carry$snowboard',
       'carry$surfboard', 'catch$frisbee', 'catch$rugby_ball',
       'catch$soccer_ball', 'cut$apple', 'cut$banana', 'cut$carrot',
       'cut$orange', 'cut_with$knife', 'cut_with$scissors',
       'drag$suitcase', 'drink_with$bottle', 'drink_with$cup',
       'drink_with$wine_glass', 'eat$apple', 'eat$banana', 'eat$broccoli',
       'eat$carrot', 'eat$hot_dog', 'eat$orange', 'hit$axe',
       'hit$baseball_bat', 'hit$hammer', 'hit$tennis_racket', 'hold$axe',
       'hold$badminton_racket', 'hold$baseball_bat', 'hold$book',
       'hold$bottle', 'hold$bowl', 'hold$cup', 'hold$fork',
       'hold$frisbee', 'hold$golf_clubs', 'hold$hammer', 'hold$knife',
       'hold$scissors', 'hold$skateboard', 'hold$skis', 'hold$snowboard',
       'hold$suitcase', 'hold$surfboard', 'hold$tennis_racket',
       'hold$toothbrush', 'hold$wine_glass', 'jump$skateboard',
       '

In [58]:
# 1. 점수 컬럼 이름들만 리스트로 뽑기
# (메타데이터 컬럼들은 제외합니다)
meta_cols = ['pair_key', 'file_name',  'action', 'object_name', 'exo_path', 'exo_w', 'exo_h','exo_area']
score_cols = [c for c in df.columns if c not in meta_cols]

# 2. 점수 데이터만 따로 꺼내서 Numpy 배열로 변환
raw_scores_matrix = df[score_cols].values.astype(float)

# 3. Softmax 일괄 계산 (행렬 연산)
# axis=1 은 '가로 방향(행)'으로 계산한다는 뜻입니다.
# (Overflow 방지를 위해 각 행의 최댓값을 빼줍니다)
max_vals = np.max(raw_scores_matrix, axis=1, keepdims=True)
exp_matrix = np.exp(raw_scores_matrix - max_vals)
probs_matrix = exp_matrix / np.sum(exp_matrix, axis=1, keepdims=True)

# 4. 결과를 담을 새로운 DataFrame 생성 (df_prob)
# 메타데이터는 그대로 복사해오고, 점수 부분만 확률로 교체합니다.
df_prob = df[meta_cols].copy() # 메타데이터 복사
prob_df_part = pd.DataFrame(probs_matrix, columns=score_cols, index=df.index) # 확률 데이터프레임
df_prob = pd.concat([df_prob, prob_df_part], axis=1) # 합치기
df_prob

,pair_key,file_name,action,object_name,exo_path,exo_w,exo_h,exo_area,beat,boxing,...,stick,stir,swing,take_photo,talk_on,text_on,throw,type_on,wash,write
0,beat$drum,beat_drum_000099.jpg,beat,drum,/home/DATA/AGD20K/Seen/trainset/exocentric/bea...,320,240,76800,1.315914e-01,2.878549e-04,...,1.188119e-05,0.002127,2.878549e-04,1.176397e-06,1.842967e-08,4.025410e-08,3.506791e-03,3.370435e-07,1.189909e-08,6.636779e-08
1,beat$drum,beat_drum_000247.jpg,beat,drum,/home/DATA/AGD20K/Seen/trainset/exocentric/bea...,640,480,307200,1.794120e-01,2.697351e-04,...,7.259821e-05,0.001993,9.414679e-04,2.670739e-05,1.035558e-06,1.316564e-07,1.066823e-03,3.614452e-06,2.039134e-07,3.395463e-06
2,beat$drum,beat_drum_000277.jpg,beat,drum,/home/DATA/AGD20K/Seen/trainset/exocentric/bea...,640,426,272640,2.171644e-01,2.019944e-02,...,6.843676e-05,0.079890,1.765007e-03,9.859250e-06,7.142016e-07,5.617631e-06,2.019944e-02,1.625515e-05,1.095264e-07,6.709304e-07
3,beat$drum,beat_drum_000389.jpg,beat,drum,/home/DATA/AGD20K/Seen/trainset/exocentric/bea...,430,640,275200,4.465411e-01,2.655230e-03,...,2.770982e-05,0.013484,1.714346e-03,7.006139e-06,1.217484e-06,1.180026e-06,6.369567e-03,4.523503e-06,3.563371e-08,1.132437e-07
4,beat$drum,beat_drum_000424.jpg,beat,drum,/home/DATA/AGD20K/Seen/trainset/exocentric/bea...,640,426,272640,3.766396e-01,4.366488e-05,...,1.606341e-05,0.000566,2.083135e-04,2.791402e-06,1.659828e-08,9.457417e-09,1.622347e-04,2.763863e-07,6.919194e-09,4.021396e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20055,write$pen,write_pen_013769.jpg,write,pen,/home/DATA/AGD20K/Seen/trainset/exocentric/wri...,640,426,272640,1.151813e-11,2.075028e-06,...,2.034286e-08,0.017899,2.144121e-09,1.159101e-08,3.672988e-11,4.167805e-05,5.298769e-06,2.836227e-06,7.526573e-10,9.772272e-01
20056,write$pen,write_pen_013824.jpg,write,pen,/home/DATA/AGD20K/Seen/trainset/exocentric/wri...,640,428,273920,1.013861e-12,6.348369e-09,...,1.507870e-09,0.000058,1.277025e-10,9.236823e-09,2.865745e-12,1.653580e-06,3.028638e-08,2.873491e-07,1.237735e-10,9.999322e-01
20057,write$pen,write_pen_013836.jpg,write,pen,/home/DATA/AGD20K/Seen/trainset/exocentric/wri...,640,426,272640,4.834142e-12,2.227900e-07,...,8.912722e-09,0.004068,3.736651e-10,2.985388e-09,1.293684e-11,1.294843e-05,1.364754e-06,2.860680e-07,3.510259e-10,9.954741e-01
20058,write$pen,write_pen_013937.jpg,write,pen,/home/DATA/AGD20K/Seen/trainset/exocentric/wri...,640,426,272640,1.523317e-11,3.891774e-07,...,1.999106e-08,0.008572,1.979383e-09,4.323371e-09,3.829151e-11,2.407755e-05,1.358363e-06,1.976406e-06,4.348128e-10,9.908120e-01


In [59]:
df_prob.pair_key.unique()

array(['beat$drum', 'boxing$punching_bag', 'brush_with$toothbrush',
       'carry$skateboard', 'carry$skis', 'carry$snowboard',
       'carry$surfboard', 'catch$frisbee', 'catch$rugby_ball',
       'catch$soccer_ball', 'cut$apple', 'cut$banana', 'cut$carrot',
       'cut$orange', 'cut_with$knife', 'cut_with$scissors',
       'drag$suitcase', 'drink_with$bottle', 'drink_with$cup',
       'drink_with$wine_glass', 'eat$apple', 'eat$banana', 'eat$broccoli',
       'eat$carrot', 'eat$hot_dog', 'eat$orange', 'hit$axe',
       'hit$baseball_bat', 'hit$hammer', 'hit$tennis_racket', 'hold$axe',
       'hold$badminton_racket', 'hold$baseball_bat', 'hold$book',
       'hold$bottle', 'hold$bowl', 'hold$cup', 'hold$fork',
       'hold$frisbee', 'hold$golf_clubs', 'hold$hammer', 'hold$knife',
       'hold$scissors', 'hold$skateboard', 'hold$skis', 'hold$snowboard',
       'hold$suitcase', 'hold$surfboard', 'hold$tennis_racket',
       'hold$toothbrush', 'hold$wine_glass', 'jump$skateboard',
       '

In [64]:
df_prob[df_prob['pair_key'] == 'cut_with$knife']

,pair_key,file_name,action,object_name,exo_path,exo_w,exo_h,exo_area,beat,boxing,...,stick,stir,swing,take_photo,talk_on,text_on,throw,type_on,wash,write
1596,cut_with$knife,cut_with_knife_000037.jpg,cut_with,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/cut...,640,480,307200,3.177321e-12,1.230128e-10,...,4.817251e-11,6.372222e-08,1.810383e-12,2.158942e-10,3.786800e-14,2.008129e-11,4.662017e-08,9.485730e-12,2.474500e-12,1.098053e-12
1597,cut_with$knife,cut_with_knife_000055.jpg,cut_with,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/cut...,428,640,273920,2.430480e-10,8.567739e-09,...,7.102905e-09,5.029138e-06,5.540081e-12,3.355175e-09,9.134050e-12,4.585980e-09,6.806198e-07,7.032828e-10,1.892860e-10,1.033467e-08
1598,cut_with$knife,cut_with_knife_000075.jpg,cut_with,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/cut...,320,240,76800,6.303809e-12,1.495034e-09,...,2.003346e-11,7.744467e-07,4.785613e-13,4.241084e-11,3.833527e-14,2.153800e-10,1.623328e-07,1.420585e-11,6.963026e-13,7.528826e-13
1599,cut_with$knife,cut_with_knife_000103.jpg,cut_with,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/cut...,640,640,409600,5.189848e-13,5.461832e-11,...,1.455529e-12,1.733156e-07,4.194040e-14,9.491240e-12,2.352643e-14,7.391785e-12,3.561562e-10,2.052625e-12,5.544531e-15,1.633058e-13
1600,cut_with$knife,cut_with_knife_000264.jpg,cut_with,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/cut...,640,426,272640,8.758025e-11,7.333005e-10,...,3.056851e-10,1.794327e-07,3.523714e-13,1.209008e-09,2.523585e-12,5.312011e-11,2.452554e-07,3.538562e-11,3.687266e-10,2.933571e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,cut_with$knife,cut_with_knife_013357.jpg,cut_with,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/cut...,640,426,272640,5.927381e-12,1.902488e-10,...,1.153917e-10,7.675183e-08,1.322577e-12,4.858172e-10,6.750759e-14,5.623743e-11,3.859324e-08,2.005207e-11,5.927381e-12,6.509955e-12
1822,cut_with$knife,cut_with_knife_013547.jpg,cut_with,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/cut...,640,480,307200,4.057079e-07,3.396951e-06,...,2.485262e-06,1.552898e-03,5.903015e-07,5.903015e-07,2.436465e-08,1.573100e-08,1.669523e-02,2.019901e-08,9.052567e-08,4.797685e-09
1823,cut_with$knife,cut_with_knife_013619.jpg,cut_with,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/cut...,400,600,240000,1.109576e-11,3.174207e-09,...,3.791050e-10,2.914556e-05,1.304654e-12,7.701984e-11,5.241266e-12,2.981891e-09,1.118947e-07,1.906259e-10,2.746227e-11,1.580345e-10
1824,cut_with$knife,cut_with_knife_013920.jpg,cut_with,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/cut...,640,437,279680,3.413710e-11,3.139190e-10,...,1.788656e-10,2.518616e-07,1.205187e-12,4.862072e-10,2.229372e-13,3.991032e-11,1.435064e-07,7.155525e-12,9.187877e-12,2.891094e-12


In [71]:
pair == 'cut_with$knife'

True

In [73]:
 df_prob[df_prob['pair_key'] == pair]

,pair_key,file_name,action,object_name,exo_path,exo_w,exo_h,exo_area,beat,boxing,...,stick,stir,swing,take_photo,talk_on,text_on,throw,type_on,wash,write
1596,cut_with$knife,cut_with_knife_000037.jpg,cut_with,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/cut...,640,480,307200,3.177321e-12,1.230128e-10,...,4.817251e-11,6.372222e-08,1.810383e-12,2.158942e-10,3.786800e-14,2.008129e-11,4.662017e-08,9.485730e-12,2.474500e-12,1.098053e-12
1597,cut_with$knife,cut_with_knife_000055.jpg,cut_with,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/cut...,428,640,273920,2.430480e-10,8.567739e-09,...,7.102905e-09,5.029138e-06,5.540081e-12,3.355175e-09,9.134050e-12,4.585980e-09,6.806198e-07,7.032828e-10,1.892860e-10,1.033467e-08
1598,cut_with$knife,cut_with_knife_000075.jpg,cut_with,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/cut...,320,240,76800,6.303809e-12,1.495034e-09,...,2.003346e-11,7.744467e-07,4.785613e-13,4.241084e-11,3.833527e-14,2.153800e-10,1.623328e-07,1.420585e-11,6.963026e-13,7.528826e-13
1599,cut_with$knife,cut_with_knife_000103.jpg,cut_with,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/cut...,640,640,409600,5.189848e-13,5.461832e-11,...,1.455529e-12,1.733156e-07,4.194040e-14,9.491240e-12,2.352643e-14,7.391785e-12,3.561562e-10,2.052625e-12,5.544531e-15,1.633058e-13
1600,cut_with$knife,cut_with_knife_000264.jpg,cut_with,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/cut...,640,426,272640,8.758025e-11,7.333005e-10,...,3.056851e-10,1.794327e-07,3.523714e-13,1.209008e-09,2.523585e-12,5.312011e-11,2.452554e-07,3.538562e-11,3.687266e-10,2.933571e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,cut_with$knife,cut_with_knife_013357.jpg,cut_with,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/cut...,640,426,272640,5.927381e-12,1.902488e-10,...,1.153917e-10,7.675183e-08,1.322577e-12,4.858172e-10,6.750759e-14,5.623743e-11,3.859324e-08,2.005207e-11,5.927381e-12,6.509955e-12
1822,cut_with$knife,cut_with_knife_013547.jpg,cut_with,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/cut...,640,480,307200,4.057079e-07,3.396951e-06,...,2.485262e-06,1.552898e-03,5.903015e-07,5.903015e-07,2.436465e-08,1.573100e-08,1.669523e-02,2.019901e-08,9.052567e-08,4.797685e-09
1823,cut_with$knife,cut_with_knife_013619.jpg,cut_with,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/cut...,400,600,240000,1.109576e-11,3.174207e-09,...,3.791050e-10,2.914556e-05,1.304654e-12,7.701984e-11,5.241266e-12,2.981891e-09,1.118947e-07,1.906259e-10,2.746227e-11,1.580345e-10
1824,cut_with$knife,cut_with_knife_013920.jpg,cut_with,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/cut...,640,437,279680,3.413710e-11,3.139190e-10,...,1.788656e-10,2.518616e-07,1.205187e-12,4.862072e-10,2.229372e-13,3.991032e-11,1.435064e-07,7.155525e-12,9.187877e-12,2.891094e-12


In [100]:
    target_df = df_prob[df_prob['pair_key'] == pair].copy()

    # 2. 점수(확률) 컬럼들만 골라내기
    # (메타데이터 컬럼 제외)
    meta_cols = ['pair_key', 'file_name', 'action', 'object_name', 'gt_score', 'softmax_score', 'gap', 'z_score', 'exo_path', 'exo_w', 'exo_h','exo_area']
    score_cols = [c for c in target_df.columns if c not in meta_cols]

    # 3. 각 행에서 최대값을 가진 '컬럼 이름' 추출 (가장 높은 확률의 행동)
    target_df['predicted_action'] = target_df[score_cols].idxmax(axis=1)

    # 4. (선택) 그때의 '확률 값'도 추출
    target_df['max_prob'] = target_df[score_cols].max(axis=1)
    target_df
    # # 5. 결과 확인
    # file_name, 정답(action), 예측(predicted_action), 확률(max_prob) 순으로 출력
    result_view = target_df[['file_name', 'action', 'predicted_action', 'max_prob','exo_area']]
    result_view.loc[:,'final_score'] =  result_view.loc[:,'max_prob'] * result_view.loc[:,'exo_area']
    
    # result_view = result_view.loc[result_view['predicted_action'].apply(lambda x : x.split('_')[0])==result_view['action'].apply(lambda x : x.split('_')[0])]
    result_view

,file_name,action,predicted_action,max_prob,exo_area,final_score
2841,hit_axe_003505.jpg,hit,cut,0.499884,1587200,7.934154e+05
2842,hit_axe_006372.jpg,hit,cut,0.499994,1500000,7.499915e+05
2843,hit_axe_006933.jpg,hit,hold,0.951732,173600,1.652207e+05
2844,hit_axe_008102.jpg,hit,throw,0.476229,250000,1.190572e+05
2845,hit_axe_008548.jpg,hit,hold,0.953410,1473400,1.404755e+06
2846,hit_axe_008636.jpg,hit,cut,0.499926,750000,3.749444e+05
2847,hit_axe_008748.jpg,hit,cut,0.488146,2250000,1.098328e+06
2848,hit_axe_009894.jpg,hit,cut,0.365012,394348,1.439419e+05
2849,hit_axe_011424.jpg,hit,cut,0.499945,2250000,1.124876e+06
2850,hit_axe_012581.jpg,hit,cut,0.499992,162500,8.124868e+04


'hit'

In [114]:
target_df.loc[[target_df[pair.split('$')[0]].idxmax()]]

,pair_key,file_name,action,object_name,exo_path,exo_w,exo_h,exo_area,beat,boxing,...,swing,take_photo,talk_on,text_on,throw,type_on,wash,write,predicted_action,max_prob
2844,hit$axe,hit_axe_008102.jpg,hit,axe,/home/DATA/AGD20K/Seen/trainset/exocentric/hit...,500,500,250000,0.000002,0.000492,...,0.002205,0.000016,3.838144e-07,0.000001,0.476229,0.000007,1.550750e-07,1.650765e-07,throw,0.476229


In [117]:
best_exo_json = {}
for pair in df_prob.pair_key.unique():
    # print(pair)
        # 1. 'hold$axe' 데이터만 필터링
    target_df = df_prob[df_prob['pair_key'] == pair].copy()

    # 2. 점수(확률) 컬럼들만 골라내기
    # (메타데이터 컬럼 제외)
    meta_cols = ['pair_key', 'file_name', 'action', 'object_name', 'gt_score', 'softmax_score', 'gap', 'z_score', 'exo_path', 'exo_w', 'exo_h','exo_area']
    score_cols = [c for c in target_df.columns if c not in meta_cols]

    # 3. 각 행에서 최대값을 가진 '컬럼 이름' 추출 (가장 높은 확률의 행동)
    target_df['predicted_action'] = target_df[score_cols].idxmax(axis=1)

    # 4. (선택) 그때의 '확률 값'도 추출
    target_df['max_prob'] = target_df[score_cols].max(axis=1)

    # 5. 결과 확인
    # file_name, 정답(action), 예측(predicted_action), 확률(max_prob) 순으로 출력
    result_view = target_df[['file_name', 'action', 'predicted_action', 'max_prob','exo_area']]
    result_view.loc[:,'final_score'] =  result_view.loc[:,'max_prob'] * result_view.loc[:,'exo_area']
    
    # result_view = result_view.loc[result_view['predicted_action']==result_view['action']]
    result_view = result_view.loc[result_view['predicted_action'].apply(lambda x : x.split('_')[0])==result_view['action'].apply(lambda x : x.split('_')[0])]
    result_view = result_view.sort_values('max_prob').reset_index(drop=True)
    if len(result_view) ==0:
        print(f"{pair} ERROR - Select action best image")
        result_view = target_df.loc[[target_df[pair.split('$')[0]].idxmax()]].reset_index(drop=True)


    file_name = result_view.loc[len(result_view)-1,'file_name']
    score = result_view.loc[len(result_view)-1,'max_prob']
    print(f"=== Result for '{pair}' : {file_name} / {np.round(score,2)} ===")
    best_exo_json[pair] = file_name
    # break


=== Result for 'beat$drum' : beat_drum_008343.jpg / 0.75 ===
=== Result for 'boxing$punching_bag' : boxing_punching_bag_013161.jpg / 0.99 ===
=== Result for 'brush_with$toothbrush' : brush_with_toothbrush_003094.jpg / 1.0 ===
=== Result for 'carry$skateboard' : carry_skateboard_012652.jpg / 0.99 ===
=== Result for 'carry$skis' : carry_skis_011502.jpg / 0.99 ===
=== Result for 'carry$snowboard' : carry_snowboard_009712.jpg / 0.95 ===
=== Result for 'carry$surfboard' : carry_surfboard_012065.jpg / 1.0 ===
=== Result for 'catch$frisbee' : catch_frisbee_001157.jpg / 0.99 ===
=== Result for 'catch$rugby_ball' : catch_rugby_ball_002776.jpg / 0.98 ===
=== Result for 'catch$soccer_ball' : catch_soccer_ball_013982.jpg / 0.62 ===
=== Result for 'cut$apple' : cut_apple_009772.jpg / 0.5 ===
=== Result for 'cut$banana' : cut_banana_013004.jpg / 0.48 ===
=== Result for 'cut$carrot' : cut_carrot_011069.jpg / 0.5 ===
=== Result for 'cut$orange' : cut_orange_002454.jpg / 0.5 ===
=== Result for 'cut_wit

In [55]:
target_df = df_prob[df_prob['pair_key'] == "stick$knife"].copy()

In [52]:
# JSON 파일로 저장
output_file = "3B_exo_best_image.json"
with open(output_file, "w", encoding="utf-16") as f:
    json.dump(best_exo_json, f, indent=4, ensure_ascii=False, sort_keys=True)


In [39]:
best_exo_json

{'beat$drum': 'hold_badminton_racket_008125.jpg'}

In [18]:
# 1. 'hold$axe' 데이터만 필터링
target_df = df_prob[df_prob['pair_key'] == 'hold$badminton_racket'].copy()

# 2. 점수(확률) 컬럼들만 골라내기
# (메타데이터 컬럼 제외)
meta_cols = ['pair_key', 'file_name', 'action', 'object_name', 'gt_score', 'softmax_score', 'gap', 'z_score', 'exo_path', 'exo_w', 'exo_h','exo_area']
score_cols = [c for c in target_df.columns if c not in meta_cols]

# 3. 각 행에서 최대값을 가진 '컬럼 이름' 추출 (가장 높은 확률의 행동)
target_df['predicted_action'] = target_df[score_cols].idxmax(axis=1)

# 4. (선택) 그때의 '확률 값'도 추출
target_df['max_prob'] = target_df[score_cols].max(axis=1)

# 5. 결과 확인
# file_name, 정답(action), 예측(predicted_action), 확률(max_prob) 순으로 출력
result_view = target_df[['file_name', 'action', 'predicted_action', 'max_prob','exo_area']]
result_view['final_score'] =  result_view['max_prob'] * result_view['exo_area']
result_view = result_view[result_view['predicted_action']==result_view['action']]
print("=== Prediction Result for 'hold$axe' ===")
result_view.sort_values('max_prob')

=== Prediction Result for 'hold$axe' ===


/tmp/ipykernel_1822003/4292024629.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_view['final_score'] =  result_view['max_prob'] * result_view['exo_area']


,file_name,action,predicted_action,max_prob,exo_area,final_score
3988,hold_badminton_racket_011723.jpg,hold,hold,0.670051,272640,182682.755534
3984,hold_badminton_racket_008125.jpg,hold,hold,0.840061,163000,136929.941861


In [21]:
result_view.loc[len(result_view)-1,'file_name']

KeyError: 1

In [ ]:
    df_prob[df_prob['pair_key'] == 'hold$axe'][['file_name','hold','hit']]*10000

,file_name,hold,hit
0,hold_axe_000384.jpghold_axe_000384.jpghold_axe...,9985.416864,0.001957
1,hold_axe_000856.jpghold_axe_000856.jpghold_axe...,0.003232,0.041593
2,hold_axe_001123.jpghold_axe_001123.jpghold_axe...,0.025535,0.000348
3,hold_axe_003505.jpghold_axe_003505.jpghold_axe...,0.018908,0.201696
4,hold_axe_006372.jpghold_axe_006372.jpghold_axe...,0.000049,0.002387
5,hold_axe_006509.jpghold_axe_006509.jpghold_axe...,9996.540377,0.000389
6,hold_axe_006933.jpghold_axe_006933.jpghold_axe...,9997.503052,0.001812
7,hold_axe_008102.jpghold_axe_008102.jpghold_axe...,9926.657870,0.013823
8,hold_axe_008636.jpghold_axe_008636.jpghold_axe...,0.002220,0.043302
9,hold_axe_009211.jpghold_axe_009211.jpghold_axe...,9992.190240,0.001783
